# 1. Submission
student_id: s201687@student.dhbw-mannheim.de
<br>This submission is to fulfill the second learning objective. The notebook consists of two parts:
<br> **first part**: Implementation of PID to OpenAI Gym Environment (Cartpole)
<br> **second part**: Implementation of a Q-Table to the frozen lake game
<br> In this notebook, I aim to provide comprehensive explanations of the concept by adding an extensive amount of comments.
<br> Why are there two parts? To fulfill the requirement of 200 - 400 lines of code.

In [1]:
# Utils
import random
import time
from IPython.display import clear_output

# Data Science
import numpy as np
import gymnasium as gym
from gymnasium.utils.play import play

In [2]:
# uncomment to install the gym package
# !pip install gymnasium
# required to display
# !pip install "gymnasium[toy-text]"
# !pip install "gymnasium[classic_control]"

## First part: PID on Cartpole

#### What is a PID?
A PID controller is a feedback control loop to regulate a system's behavior. PID stands for "proportional-integral-derivative," which refers to the three components that make up the controller.

**Proportional (P)**: This component of the controller produces an output that is proportional to the current error signal. The error signal is the difference between the desired setpoint and the current measured value of the system output. The P component helps to reduce the error signal by producing a corrective action that is proportional to the current error.

**Integral (I)**: This component of the controller produces an output that is proportional to the accumulated error signal over time. The I component helps to eliminate steady-state errors by continuously adjusting the control signal based on the system's history of errors.

**Derivative (D)**: This component of the controller produces an output that is proportional to the rate of change of the error signal. The D component helps to dampen the system's response to sudden changes in the error signal and reduce overshoot and oscillations.

In [3]:
# run the following cell to play Cartpole as human
# a = left, d = right
keys = { "a": 0, "d": 1 }
env = gym.make("CartPole-v1", render_mode="rgb_array")
# noop=0 means default action is move left
play(env, keys_to_action=keys, noop=0)

In [5]:
# Change render_mode to "human" if you want to see the agent playing
env = gym.make("CartPole-v1", render_mode="rgb_array")

In [4]:
class PD:
    def __init__(self, kp, kd, goal):
        self.kp = kp
        self.kd = kd
        self.goal = goal
        self.last_error = 0

    def observe(self, x):
        error = self.goal - x
        d_error = error - self.last_error
        self.last_error = error
        return self.kp * error + self.kd * d_error

#### What are kp and kd?
The kp value determines the strength of the proportional term in the controller. In this case, it is set to 5. A higher kp value would make the controller respond more strongly to errors (deviations from the goal) and could result in faster but more oscillatory control.

The kd value determines the strength of the derivative term in the controller. In this case, it is set to 100. The derivative term helps to dampen the control response by taking into account the rate of change of the error. A higher kd value would make the controller respond more strongly to changes in error, which could result in faster but more overshoot-prone control.

In [6]:
controller = PD(kp=5, kd=100, goal=0)
observation, info = env.reset()

# Run the game for 1000 episode
for episode in range(1000):
    # Get the pole angle from the current observation
    pole_angle = observation[2]
    # Use the PD controller to calculate the control output
    control_output = controller.observe(pole_angle)
    # Determine the action based on the control output
    action = 1 if control_output < 0 else 0
    
    # Take the action in the environment and get the new observation, reward, termination, and info values
    observation, reward, terminated, truncated, info = env.step(action)
    
    # If the game has ended, reset the environment to start a new game
    if terminated or truncated:
        observation, info = env.reset()

env.close()

## Second part: Frozen Lake game
In the FrozenLake game, the goal of the agent is to navigate a grid of frozen lake tiles, represented as a two-dimensional array of S, F, H, and G characters, where 
<br>S = Starting point
<br>F = Frozen titles
<br>H = Holes that the agent must avoid
<br>G = Goal
<br>The agent can take four possible actions in each state: move up, move down, move left, or move right. The agent receives a reward of 1 for reaching the goal, a reward of 0 for stepping on a frozen tile, and a reward of -1 for falling into a hole. The episode ends when the agent reaches the goal or falls into a hole.
<br>For more resource: https://www.gymlibrary.dev/environments/toy_text/frozen_lake/

In [7]:
# Creating the OpenAI Gym environment "FrozenLake-v1"
# map_name => map will be a 4x4 grid
# is_slippery = True => Move in intended direction with a probability of 1/3
# change render_mode to "human" if you want to see the agent plays
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, render_mode="rgb_array")

In [8]:
# number of columns
action_space_size = env.action_space.n

# number of rows
state_space_size = env.observation_space.n

# Creating a Q-table with (column, rows) and initializing all values to 0
q_table = np.zeros((state_space_size, action_space_size))
print(q_table)

[[0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]
 [0. 0. 0. 0.]]


In [9]:
# 16 because there are 16 cells (4x4)
# 4 because there are 4 actions (left, right, up, down)
print(q_table.shape)

(16, 4)


#### What is a Q-Table?
A Q-table is a table used in reinforcement learning to represent a policy in tabular form. It maps a state-action pair to an expected cumulative reward, which is the expected cumulative reward that an agent will receive if it takes the specified action in the given state and then follows its policy to select actions thereafter. Over time, the Q-table converges to the optimal values, indicating the optimal policy for the agent to follow. This technique is applicable for our use case but is infeasible for larger and more complex environments where the number of possible states and actions is very large or infinite. More sophisticated methods like function approximation or deep learning are used to represent the policy.

#### How does Q-Table apply to our game?
The Q-table for the FrozenLake game has dimensions (number of states, number of actions) and is initialized with all values set to 0. During training, the agent uses an exploration-exploitation strategy to select actions. The Q-values in the table are updated after each action using the Bellman equation, which incorporates the reward received, the Q-value of the next state, and a discount factor that determines the importance of future rewards.

Once the agent has been trained, it can use the Q-table to select actions in a greedy manner, choosing the action with the highest Q-value in each state. The performance of the agent is typically evaluated by measuring the percentage of successful episodes, or the average cumulative reward over a set of episodes.

#### What is an exploration-exploitation strategy?
Exploration-exploitation is a strategy that refers to the balance between choosing actions that are already known to be good (exploitation) and trying out new actions to gain more information about the environment (exploration).
<br>Initially the agent has limited information about the environment, and it needs to explore to learn which actions lead to higher rewards. Once the agent has gained some knowledge about the environment, it can use this knowledge to exploit the actions that are already known to be good. Essentially it's a trade-off. If the agent only exploits its current knowledge, it may miss better actions that it has not yet tried. On the other hand, if the agent only explores new actions, it may waste time and miss out on immediate rewards.

In [10]:
num_episodes = 10000
max_steps_per_episode = 100

learning_rate  = 0.1
discount_rate = 0.99

# 1 -> agent should explore first
exploration_rate = 1
max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.002

- **num_episodes**: An episode in FrozenLake is a playthrough of the game starting from initial state (the starting point of the agent on the frozen lake) and ending when the agent reaches a terminal state (the goal tile or a hole)
- **max_steps_per_episode**: Maximum number of steps during an episode
- **learning_rate**: Typical learning rate in ml, it controls how much the agent updates its estimates of the values of state-action pairs based on the rewards it receives. A high learning rate means that the agent gives more weight to the most recent rewards when updating its estimates, while a low learning rate means that the agent relies more on past rewards.
- **discount_rate**: controls how much the agent values future rewards compared to immediate rewards. Specifically, the discount rate determines the factor by which future rewards are discounted relative to immediate rewards. A high discount rate means that the agent values future rewards more highly than immediate rewards, while a low discount rate means that the agent values future rewards less. The value ranges between 0 to 1, so in our case 0.99 is a pretty high discount rate
- **exploration_rate**: controls the trade-off between exploration and exploitation in the agent's behavior. In other words, the exploration rate determines how much the agent should explore the environment (i.e., try new actions) versus exploiting what it has already learned (i.e., selecting actions that it knows lead to high rewards).
- **exploration_decay_rate**: ensures that the exploration rate decreases slowly over time, allowing the agent to gradually shift from exploration to exploitation as it learns more about the environment.

In [11]:
# empty list to store the rewards for each episode
rewards_all_episodes = []

# Q-Learning algorithm
for episode in range(num_episodes):
    # reset environment
    state = env.reset()[0]
    
    # set done to False indicating the episode is not over yet
    done = False
    # reset total reward for current episode
    rewards_current_episode = 0
    
    for step in range(max_steps_per_episode):
        
        # Exploration-exploitation trade-off
        # generate a random number between 0 and 1
        exploration_rate_threshold = random.uniform(0, 1)
        # if the random number is greater than exploration rate
        if exploration_rate_threshold > exploration_rate:
            # Exploit and select the action with the highest Q-value for current state
            action = np.argmax(q_table[state,:])
        else:
            # Explore and select a random action
            action = env.action_space.sample()
        
        obs, reward, done, truncated, info = env.step(action)
        
        # Update Q-table for Q(s, a)
        q_table[state, action] = q_table[state, action] * (1 - learning_rate) + \
        learning_rate * (reward + discount_rate * np.max(q_table[obs, :]))
        
        # Update state
        state = obs
        # Add reward to current episode
        rewards_current_episode += reward
        
        if done == True:
            break
    
    # exploration rate decay
    exploration_rate = min_exploration_rate + \
    (max_exploration_rate - min_exploration_rate) * np.exp(-exploration_decay_rate * episode)
    
    rewards_all_episodes.append(rewards_current_episode)

# Calculate and print the average reward per thousand episodes
rewards_per_thousand_episodes = np.split(np.array(rewards_all_episodes), num_episodes/1000)
count = 1000

print("**Average reward per thousand episodes**\n")
for r in rewards_per_thousand_episodes:
    print(f"count : {str(sum(r/1000))}")
    count += 1000

# Print updated Q-table
print("\n\n**Q-Table**\n")
print(q_table)

**Average reward per thousand episodes**

count : 0.11500000000000009
count : 0.47300000000000036
count : 0.6490000000000005
count : 0.6880000000000005
count : 0.6750000000000005
count : 0.6980000000000005
count : 0.6560000000000005
count : 0.6700000000000005
count : 0.6960000000000005
count : 0.6680000000000005


**Q-Table**

[[0.55595145 0.47183209 0.48521633 0.50071495]
 [0.2584281  0.26511069 0.18060026 0.45908729]
 [0.3809588  0.25326703 0.20935374 0.25136617]
 [0.20629013 0.00410903 0.00130697 0.03373852]
 [0.57179473 0.28503557 0.36373593 0.39380625]
 [0.         0.         0.         0.        ]
 [0.17581237 0.11501583 0.29225537 0.11868138]
 [0.         0.         0.         0.        ]
 [0.31512028 0.40848283 0.45363947 0.60533011]
 [0.43986605 0.66057858 0.52260562 0.31519539]
 [0.65670751 0.42471528 0.45936102 0.32433251]
 [0.         0.         0.         0.        ]
 [0.         0.         0.         0.        ]
 [0.5189993  0.50365942 0.77609729 0.44860942]
 [0.70945535 

The reward for each episode is either 1 or 0. So if we look at the number, for the first 1000 episodes, the agent was winning only 12% of the time. By the last thousand episodes, it was winning 67% all the time

obs, reward, done, truncated, info = env.step(action)
<br> example output: (4, 0.0, False, False, {'prob': 0.3333333333333333})
- **obs**: The observation/state after the action is taken
- **reward**: The reward obtained for taking the action.
- **done**: It is a boolean value that indicates whether the episode is terminated or not. If it is True, then the current episode is terminated.
- **truncated**: It is a boolean value that indicates whether the episode was truncated due to reaching the maximum number of steps or not.
- **info**: Additional information about the environment's state. It is usually not used in the learning algorithm.

I add more comment to this line because there are are differences between the old and new version of gym.
source: https://stackoverflow.com/questions/52950547/getting-error-valueerror-too-many-values-to-unpack-expected-5

In [12]:
# define env with render_mode "human" to display agent playing
env = gym.make("FrozenLake-v1", map_name="4x4", is_slippery=True, render_mode="human")

# Loop through episodes
for episode in range(2):
    # Reset environment
    state = env.reset()[0]
    print(f"EPISODE {episode + 1}")
    time.sleep(1)

    # Reset episode-specific variables
    done = False
    step = 0

    # Loop through steps in the episode
    while not done and step < max_steps_per_episode:
        clear_output(wait=True)
        env.render()
        time.sleep(0.3)

        # Choose action based on Q-table
        action = np.argmax(q_table[state, :])

        # Take action and observe new state and reward
        obs, reward, done, truncated, info = env.step(action)

        # Update state
        state = obs

        # Increment step counter
        step += 1

    # Print final message and pause before moving on to next episode
    clear_output(wait=True)
    env.render()
    if reward == 1:
        print("You reached the goal!")
    else:
        print("You fell through the hole!")
    time.sleep(3)

# Close environment
env.close()

You reached the goal!


If it doesn't get displayed, check this thread: https://github.com/openai/gym/issues/762
<br> or try this: https://pypi.org/project/gym-notebook-wrapper/

ressource to create the pid: https://statusfailed.com/blog/2022-12-08-pid-controller-in-the-openai-gym/

The creation of the Frozenlake game was created with the help from reinforcement learning playlist from deeplizard: https://youtube.com/playlist?list=PLZbbT5o_s2xoWNVdDudn51XM8lOuZ_Njv